Spark DataFrames are the workhouse and main way of working with Spark and Python post Spark 2.0. DataFrames act as powerful versions of tables, with rows and columns, easily handling large datasets. The shift to DataFrames provides many advantages:

* A much simpler syntax
* Ability to use SQL directly in the dataframe
* Operations are automatically distributed across RDDs

In [2]:
from pyspark.sql import SparkSession

In [3]:
df = spark.read.csv('/FileStore/tables/sales_info.csv',inferSchema=True,header=True)

Spark DataFrame Basics

In [5]:
df.show()

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 GOOG| Sam|200.0|
 GOOG|Charlie|120.0|
 GOOG| Frank|340.0|
 MSFT| Tina|600.0|
 MSFT| Amy|124.0|
 MSFT|Vanessa|243.0|
 FB| Carl|870.0|
 FB| Sarah|350.0|
 APPL| John|250.0|
 APPL| Linda|130.0|
 APPL| Mike|750.0|
 APPL| Chris|350.0|
+-------+-------+-----+

In [6]:
df.printSchema()

root
-- Company: string (nullable = true)
-- Person: string (nullable = true)
-- Sales: double (nullable = true)

In [7]:
df.columns

Out[ 5 ]: ['Company', 'Person', 'Sales']

In [8]:
df.describe()

Out[ 6 ]: DataFrame[summary: string, Company: string, Person: string, Sales: string]

In [9]:
df.select('Company').show()

+-------+
Company|
+-------+
 GOOG|
 GOOG|
 GOOG|
 MSFT|
 MSFT|
 MSFT|
 FB|
 FB|
 APPL|
 APPL|
 APPL|
 APPL|
+-------+

In [10]:
df.select(['Company','Person']).show()

+-------+-------+
Company| Person|
+-------+-------+
 GOOG| Sam|
 GOOG|Charlie|
 GOOG| Frank|
 MSFT| Tina|
 MSFT| Amy|
 MSFT|Vanessa|
 FB| Carl|
 FB| Sarah|
 APPL| John|
 APPL| Linda|
 APPL| Mike|
 APPL| Chris|
+-------+-------+

In [11]:
df.filter("Sales<500").show()

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 GOOG| Sam|200.0|
 GOOG|Charlie|120.0|
 GOOG| Frank|340.0|
 MSFT| Amy|124.0|
 MSFT|Vanessa|243.0|
 FB| Sarah|350.0|
 APPL| John|250.0|
 APPL| Linda|130.0|
 APPL| Chris|350.0|
+-------+-------+-----+

In [12]:
df.filter("Sales<500").select('Person').show()

+-------+
 Person|
+-------+
 Sam|
Charlie|
 Frank|
 Amy|
Vanessa|
 Sarah|
 John|
 Linda|
 Chris|
+-------+

In [13]:
df.filter( (df["Sales"] < 500) & (df['Company'] == 'FB') ).show()

+-------+------+-----+
Company|Person|Sales|
+-------+------+-----+
 FB| Sarah|350.0|
+-------+------+-----+

In [14]:
df.filter( (df["Sales"] < 500) & ~(df['Company'] == 'FB') ).show()

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 GOOG| Sam|200.0|
 GOOG|Charlie|120.0|
 GOOG| Frank|340.0|
 MSFT| Amy|124.0|
 MSFT|Vanessa|243.0|
 APPL| John|250.0|
 APPL| Linda|130.0|
 APPL| Chris|350.0|
+-------+-------+-----+

In [15]:
df.groupBy("Company").mean().show()

+-------+-----------------+
Company| avg(Sales)|
+-------+-----------------+
 APPL| 370.0|
 GOOG| 220.0|
 FB| 610.0|
 MSFT|322.3333333333333|
+-------+-----------------+

In [16]:
df.groupBy("Company").count().show()

+-------+-----+
Company|count|
+-------+-----+
 APPL| 4|
 GOOG| 3|
 FB| 2|
 MSFT| 3|
+-------+-----+

In [17]:
df.groupBy("Company").max().show()

+-------+----------+
Company|max(Sales)|
+-------+----------+
 APPL| 750.0|
 GOOG| 340.0|
 FB| 870.0|
 MSFT| 600.0|
+-------+----------+

In [18]:
df.groupBy("Company").sum().show()

+-------+----------+
Company|sum(Sales)|
+-------+----------+
 APPL| 1480.0|
 GOOG| 660.0|
 FB| 1220.0|
 MSFT| 967.0|
+-------+----------+

Check out this link for more info on other methods:
http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark-sql-module

Not all methods need a groupby call, instead you can just call the generalized .agg() method, that will call the aggregate across all rows in the dataframe column specified. It can take in arguments as a single column, or create multiple aggregate calls all at once using dictionary notation.

For example:

In [20]:
df.agg({'Sales':'max'}).show()

+----------+
max(Sales)|
+----------+
 870.0|
+----------+

In [21]:
df.orderBy("Sales").show()

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 GOOG|Charlie|120.0|
 MSFT| Amy|124.0|
 APPL| Linda|130.0|
 GOOG| Sam|200.0|
 MSFT|Vanessa|243.0|
 APPL| John|250.0|
 GOOG| Frank|340.0|
 FB| Sarah|350.0|
 APPL| Chris|350.0|
 MSFT| Tina|600.0|
 APPL| Mike|750.0|
 FB| Carl|870.0|
+-------+-------+-----+

In [22]:
df.orderBy(df["Sales"].desc()).show()

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 FB| Carl|870.0|
 APPL| Mike|750.0|
 MSFT| Tina|600.0|
 FB| Sarah|350.0|
 APPL| Chris|350.0|
 GOOG| Frank|340.0|
 APPL| John|250.0|
 MSFT|Vanessa|243.0|
 GOOG| Sam|200.0|
 APPL| Linda|130.0|
 MSFT| Amy|124.0|
 GOOG|Charlie|120.0|
+-------+-------+-----+

#### Using SQL

To use SQL queries directly with the dataframe, you will need to register it to a temporary view:

In [24]:
df.createOrReplaceTempView("sales")

In [25]:
sql_results = spark.sql("SELECT * FROM sales")

In [26]:
sql_results.show()

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 GOOG| Sam|200.0|
 GOOG|Charlie|120.0|
 GOOG| Frank|340.0|
 MSFT| Tina|600.0|
 MSFT| Amy|124.0|
 MSFT|Vanessa|243.0|
 FB| Carl|870.0|
 FB| Sarah|350.0|
 APPL| John|250.0|
 APPL| Linda|130.0|
 APPL| Mike|750.0|
 APPL| Chris|350.0|
+-------+-------+-----+

In [27]:
spark.sql("SELECT * FROM sales WHERE Company = 'FB'").show()

+-------+------+-----+
Company|Person|Sales|
+-------+------+-----+
 FB| Carl|870.0|
 FB| Sarah|350.0|
+-------+------+-----+

There are a variety of functions you can import from pyspark.sql.functions. Check out the documentation for the full list available:

http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions

In [29]:
from pyspark.sql.functions import countDistinct, avg,stddev

In [30]:
df.select(countDistinct("Sales")).show()

+---------------------+
count(DISTINCT Sales)|
+---------------------+
 11|
+---------------------+

In [31]:
df.select(avg('Sales')).show()

+-----------------+
 avg(Sales)|
+-----------------+
360.5833333333333|
+-----------------+

In [32]:
df.select(stddev("Sales")).show()

+------------------+
stddev_samp(Sales)|
+------------------+
250.08742410799007|
+------------------+

In [33]:
from pyspark.sql.functions import format_number

In [34]:
sales_std.select(format_number('std',2)).show()

+---------------------+
format_number(std, 2)|
+---------------------+
 250.09|
+---------------------+